# Finite elements in 1D

In this worksheet, we use NGSolve to perform finite element tasks using a  simple  one-dimensional mesh. We focus on 1D to see some of the intrinsics of finite elements.

In [1]:
import ngsolve as ng
import netgen.gui             # brings up a Netgen window
import netgen.meshing as ngm  # meshing module
from netgen.csg import Pnt    # Points (or mesh vertices)

NGSolve optimizes its user interface to two and three-dimensional meshes (as most users do not want to bother with 1D). In 2D and 3D, meshes can be automatically generated by the mesh generator `Netgen` (included with the NGSolve distribution). However, unlike the two and three-dimensional case, in one-dimension, we will have to "make" the one-dimensional mesh ourselves.

### Make a 1D mesh

In [2]:
def make1dmesh(N):
    """Subdivide [0, 1] into a mesh of N elements"""
    
    m = ngm.Mesh()
    m.dim = 1
    pnums = []
    
    for i in range(0, N+1):
        pnums.append(m.Add(ngm.MeshPoint(Pnt(i/N, 0, 0))))
    for i in range(0, N):
        m.Add(ngm.Element1D([pnums[i], pnums[i+1]], index=1))

    m.Add(ngm.Element0D(pnums[0],  index=1)) # left bc marked 1
    m.Add(ngm.Element0D(pnums[-1], index=2)) # right bc marked 2
    m.SetBCName(0, 'left')
    m.SetBCName(1, 'right')

    return m

The above makes a basic `Netgen` mesh object using `MeshPoint` and `Element1D` objects. Next, we convert it to an `NGSolve` mesh which can be passed as argument to NGSolve's finite element functionalities.

In [3]:
N = 5   # the number of mesh elements
m = make1dmesh(N)
mesh = ng.Mesh(m)

### Lagrange finite element space in 1D

The next steps use functionalities that are common for any finite element space (be it in 1D, 2D, or 3D) implemented in NGSolve.  The syntax `H1` is understandable since Lagrange finite elements are used to approximate weak formulations in the Sobolev space $H^1$. 

In [ ]:
V = ng.H1(mesh, order=1)  # Lowest order is 1 (linear)

### Interpolation into the finite element space

Functions can be interpolated into this finite element space using the `Set` method.  To declare functions in terms of coordinates (just $x$ in 1D, $x, y, z$ in 3D etc), these coordinates are available as NGSolve `CoefficientFunction` objects.

In [ ]:
from ngsolve import x     

v = ng.GridFunction(V, 'myfun')
v.Set(x * x)

Now `v` is a computational representation of a finite element function in the Lagrange finite element space `V`. We can visualize this function in many ways.   1D visualization is a bit clumsy in NGSolve. But you can do it as follows.

In [ ]:
from ngsolve.internal import viewoptions, visoptions

# you might have to tweak the defaults
viewoptions.drawcolorbar = False
viewoptions.drawedges = 1
visoptions.autoscale = False
visoptions.deformation = True
visoptions.scaledeform1 = 0.5 

In [ ]:
ng.Draw(v)   # the graph of `v` should be visible in Netgen window

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pts = [vtx.point[0] for vtx in mesh.vertices]
plt.plot(pts, np.array(v.vec), '.-')
plt.xlabel('x'); plt.ylabel('$x^2$');

### Basis expansion

When `v` is expressed in term of finite element basis of hat functions $\{\psi_i\}$, 
$$
v = \sum_i v_i \psi_i
$$
the vector of coefficients $v_i$ in this basis expansion
gives all information contained in `v`. This vector can be printed out.

In [ ]:
print(v.vec)

Note that `v` and `v.vec` are objects of different classes: the latter is suitable for linear algebra, while the former models a function on the domain.

In [ ]:
type(v), type(v.vec)

### Visualizing  shape functions

In order to visualize a global basis function, it's enough to set one entry in the vector to 1 and all others to zero.

In [ ]:
v.vec[:] = 0
v.vec[3] = 1
ng.Draw(v)

### Local element matrix

The local element matrix of the bilinear form 
$$
a(u, w) = \int_0^1 u' v'
$$
can be printed out, after making the bilinear form, as follows.

In [ ]:
from ngsolve import SymbolicBFI, grad

u = V.TrialFunction()
w = V.TestFunction()

a = ng.SymbolicBFI(grad(u) * grad(w))

Next, from the following list of all mesh elements, we select an element `el` whose local element matrix we want.

In [ ]:
els = list(V.Elements())
for el in els:
    print(ng.ElementId(el))   

In [ ]:
el = els[2]  # element selected

# calculate & print out the local element matrix
elmat = a.CalcElementMatrix(el.GetFE(), el.GetTrafo())
print(elmat)

### Connectivity data structure

For assembly, NGSolve uses the mapping between local dofs and global dofs. This mapping can be printed out for any element. Here is an example:

In [ ]:
print(el.dofs)

### Essential boundary conditions 

Incorporating essential boundary conditions into a finite element space is done using the `dirichlet` argument.

In [ ]:
Vo = ng.H1(mesh, order=1, dirichlet='left|right') 
vo = ng.GridFunction(Vo, 'myfuno')
vo.Set(1, ng.BND)
ng.Draw(vo)

Remember that *we* named the boundaries when we created the mesh, and the same names are used in the `dirichlet` argument. The names can be retrieved by the following method. 

In [ ]:
mesh.GetBoundaries()

### Solve a problem

Let us solve  the BVP
$$
-u'' = 1
$$
with zero Dirichlet boundary conditions at left and right boundaries. 

In [ ]:
from ngsolve import dx

u, v = Vo.TnT()

a = ng.BilinearForm(Vo)
a += grad(u) * grad(v) * dx

b = ng.LinearForm(Vo)
b += 1 * v * dx

a.Assemble()
b.Assemble()

uh = ng.GridFunction(Vo, 'mysolution')
uh.vec.data = a.mat.Inverse(Vo.FreeDofs()) * b.vec
ng.Draw(uh)